# Pricing Project
## Identify Exclusions
Need to identify exclusions for:
* Outliers
* Customers
* Items

In [1]:
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', 100)

In [2]:
loc = Path.cwd().parent / 'data' / 'generated' / 'invoice_data_0_raw.csv'
inv_df = pd.read_csv(loc)

C:\Users\franc\AppData\Local\Temp\ipykernel_9716\742728568.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  inv_df = pd.read_csv(loc)


In [3]:
inv_df.head()

,whse,orderno,ordersuf,lineno,invoicedt,custno,custname,item,itemdesc,unit,unitconv,units,unitcost,replcost,unitprice,netamt,returnfl,transtype,prod_type,prodcat,cat_descrip,rowpointer,priceorigcd,pdrecno,vendno,arpvendno,xcost_adj,GP$,Margin,month
0,101,933,0,1,2022-02-09,107113,K & S,PAICADS2,CADS-2 DUCT SEALANT GRAY 2 GAL,EA,1.0,1.0,20.20459,21.700,26.80,26.80,0,CS,stocked,6600,ADHESIVES/SEALANTS/TAPES,d41fa94f-2390-e6b4-9014-1a70708d5123,2,57983,16632.0,0.0,20.20459,6.59541,0.246097,2022-02
1,101,933,0,2,2022-02-09,107113,K & S,DIVB503,"B-503 2"" CHIP BRUSH",PC,1.0,1.0,0.25046,0.260,1.01,1.01,0,CS,stocked,6850,EQUIPMENT AND ACC,e5691b06-5a13-6685-9014-1a70d0a6f5a0,7,3285,12773.0,0.0,0.25046,0.75954,0.752020,2022-02
2,101,911000109,0,1,2022-01-27,100151,BD OF ED GARFIELD,GSS62,16GA. 48X96 GALV STEEL,SH,1.0,24.0,98.09914,117.260,148.57,3565.68,0,SO,stocked,5300,"SHEETS, GALVANIZED STEEL",b12fdaef-1b54-6791-8514-397f4855a7a5,O,0,15105.0,0.0,2354.37936,1211.30064,0.339711,2022-01
3,101,911000109,0,2,2022-01-27,100151,BD OF ED GARFIELD,GSS62,16GA. 48X96 GALV STEEL,SH,1.0,10.0,98.09914,96.824,148.57,1485.70,0,SO,stocked,5300,"SHEETS, GALVANIZED STEEL",b12fdaef-1b54-6791-8514-397f98c915aa,O,0,15105.0,0.0,980.99140,504.70860,0.339711,2022-01
4,101,911000527,0,2,2022-03-15,109762,DAP HVAC - 154 HAMILTON ST,ICP9511,N92ESN0601412A 92% 60M FURNACE,EA,1.0,5.0,759.02801,819.540,710.00,3550.00,0,SO,stocked,4060,ICP 92%+ GAS FURNACES,8687f4d8-b12e-279a-8514-25b170dfd11e,O,0,14204.0,0.0,3795.14005,-245.14005,-0.069054,2022-03


In [4]:
# add Exclusion column to raw file
inv_df['Exclusion'] = None

## Items
* Nonstock
* Special Order
* Labor Codes

In [5]:
# Remove any Nonstock Items
null_criteria = inv_df['Exclusion'].isnull()
ns_criteria = inv_df['prod_type'] == 'nonstock'
inv_df.loc[null_criteria & ns_criteria, 'Exclusion'] = 'Item - nonstock'

In [6]:
# Remove any Special Order Items
null_criteria = inv_df['Exclusion'].isnull()
so_criteria = inv_df['prod_type'] == 'special order'
inv_df.loc[null_criteria & so_criteria, 'Exclusion'] = 'Item - special order'

In [7]:
# Remove Labor Code Items
null_criteria = inv_df['Exclusion'].isnull()
labor_criteria = inv_df['prodcat'] == 8888
inv_df.loc[null_criteria & labor_criteria, 'Exclusion'] = 'Item - labor'

In [8]:
inv_df['prod_type'].unique()

array(['stocked', 'nonstock', 'special order'], dtype=object)

## Customers
* Accounts that don't have custname filled in, assuming these are inactive
* Customer with XXX in the name
* Any customers that appear in the ../data/inputs/customer_exclusion.xlsx file

In [9]:
# fill in missing customer names with 'Inactive'
inv_df['custname'] = inv_df['custname'].fillna('INACTIVE')

In [10]:
# Remove any inactive customers
null_criteria = inv_df['Exclusion'].isnull()
inactive_acct_criteria = inv_df['custname'].str.contains('INACTIVE')
inv_df.loc[null_criteria & inactive_acct_criteria, 'Exclusion'] = 'Customer - inactive acct'

In [11]:
# Remove any customers with 'XXX' in the name
null_criteria = inv_df['Exclusion'].isnull()
xxx_acct_criteria = inv_df['custname'].str.contains('XXX')
inv_df.loc[null_criteria & xxx_acct_criteria, 'Exclusion'] = 'Customer - xxx acct'

In [12]:
# Remove any customers in the ../data/customer_exclusions.xlsx file
loc = Path.cwd().parent / 'data' / 'inputs' / 'customer_exclusions.xlsx'
cust_ex_df = pd.read_excel(loc, engine='openpyxl')
cust_exclusion_map = {i[0]:i[1] for i in cust_ex_df.to_dict('split')['data']}
inv_df['Exclusion'] = inv_df['Exclusion'].combine_first(inv_df['custno'].map(cust_exclusion_map))

## Outliers
* Quantity Shipped < 0
* Unit Cost = 0
* Unit Price = 0
* Net Amount <= 0
* Gross Profit Dollars <= 0
* Gross Profit Percent < 0.5% or > 80%

In [13]:
# Remove any lines with negative quantity shipped
null_criteria = inv_df['Exclusion'].isnull()
qty_ship_criteria = inv_df['units'] < 0
inv_df.loc[null_criteria & qty_ship_criteria, 'Exclusion'] = 'Outlier - negative quantity'

In [14]:
# Remove any lines with zero unit cost
null_criteria = inv_df['Exclusion'].isnull()
unit_cost_criteria = inv_df['unitcost'] == 0
inv_df.loc[null_criteria & unit_cost_criteria, 'Exclusion'] = 'Outlier - zero cost'

In [15]:
# Remove any lines with zero unit price
null_criteria = inv_df['Exclusion'].isnull()
unit_price_criteria = inv_df['unitprice'] == 0
inv_df.loc[null_criteria & unit_price_criteria, 'Exclusion'] = 'Outlier - zero price'

In [16]:
# Remove any lines with net amount less than or equal to zero
null_criteria = inv_df['Exclusion'].isnull()
unit_price_criteria = inv_df['netamt'] <= 0
inv_df.loc[null_criteria & unit_price_criteria, 'Exclusion'] = 'Outlier - zero net amount'

In [17]:
# Remove any lines with negative gross profit dollars
null_criteria = inv_df['Exclusion'].isnull()
gp_dollar_criteria = inv_df['GP$'] <= 0
inv_df.loc[null_criteria & gp_dollar_criteria, 'Exclusion'] = 'Outlier - negative margin dollars'

In [18]:
# Remove any lines with negative gross profit dollars
null_criteria = inv_df['Exclusion'].isnull()
gp_pct_low_criteria = inv_df['Margin'] < .005
gp_pct_high_criteria = inv_df['Margin'] > .8
inv_df.loc[null_criteria & (gp_pct_low_criteria | gp_pct_high_criteria), 'Exclusion'] = 'Outlier - high/low margin pct'

## Max Gurtman: whse = 601 or 602

In [19]:
# Remove any lines with whse = 601
null_criteria = inv_df['Exclusion'].isnull()
Max_Gurtman_601_criteria = inv_df['whse'] == 601
inv_df.loc[null_criteria & Max_Gurtman_601_criteria, 'Exclusion'] = 'Warehouse - 601'

In [20]:
# Remove any lines with whse = 602
null_criteria = inv_df['Exclusion'].isnull()
Max_Gurtman_602_criteria = inv_df['whse'] == 602
inv_df.loc[null_criteria & Max_Gurtman_602_criteria, 'Exclusion'] = 'Warehouse - 602'

## Summary

In [22]:
inv_df.loc[inv_df['prod_type'] == 'nonstock'].groupby(by='vendno', as_index=False).agg({'netamt':'sum'}).to_clipboard(index=False)

In [21]:
inv_df['Exclusion'].value_counts()

Item - special order                 26461
Outlier - negative quantity          12452
Item - nonstock                      10751
Outlier - zero net amount             6613
Customer - cash                       4633
Outlier - negative margin dollars     3478
Customer - fake acct                  1136
Outlier - zero cost                   1054
Outlier - high/low margin pct          642
Item - labor                           592
Outlier - zero price                   545
Customer - employee acct               363
Customer - xxx acct                    150
Customer - inactive acct                10
Customer - delete                       10
Warehouse - 602                          4
Name: Exclusion, dtype: int64

In [22]:
inv_df[['Exclusion','netamt']].groupby(by='Exclusion').sum('netamt')

,netamt
Exclusion,
Customer - cash,865950.93
Customer - delete,1638.62
Customer - employee acct,20126.16
Customer - fake acct,67650.48
Customer - inactive acct,4136.96
Customer - xxx acct,21602.94
Item - labor,28250.08
Item - nonstock,16633467.86
Item - special order,18936081.29


In [23]:
inv_df[['Exclusion']].isna().any(axis=1).sum()

264960

## Save to csv

In [24]:
loc = Path.cwd().parent / 'data' / 'generated' / 'invoice_data_1_exclusions_labeled.csv'
inv_df.to_csv(loc, index=False)

In [25]:
inv_df.loc[inv_df['Exclusion'] == 'Customer - inactive acct']['netamt'].sum()

4136.96

In [26]:
inv_df.loc[inv_df['Exclusion'] == 'Warehouse - 602']['netamt'].sum()

243.78000000000003

In [27]:
sum(inv_df['netamt'])

138958096.3599987